<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#000066;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">

<h1 style="padding: 15px;
              color:white;">Tabular Playground Series - July 2021 </h1>

</div>

<p style="padding: 4 px;color:black;">
        <h3 style="padding: 3px;color:black;" > Data Features </h3>
        <h3 style="padding: 3px;color:black;" > Data Visualization </h3>        
        <h3 style="padding: 3px;color:black;"> Model Training </h3>
        <h3 style="padding: 3px;color:black;"> Submission </h3>
</p>

#### *Reference & Credits:* 
https://www.kaggle.com/andy6804tw/catboost-13feature-cross-validation
<br>
https://www.kaggle.com/anjalianupam/tps-july-eda
<br>
https://www.kaggle.com/hiro5299834/tps-jul-2021-votingregressor-leaveonegroupout
<br>
https://www.kaggle.com/alexryzhkov/tps-lightautoml-baseline-with-pseudolabels

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd  
import seaborn as sns 

from sklearn.metrics import mean_squared_log_error

from sklearn.model_selection import LeaveOneGroupOut
from catboost import CatBoostRegressor
from datetime import datetime
from skmultilearn.problem_transform import ClassifierChain

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
# load submission
preds = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
# Dropping the last row which is 2011-01-01 00:00:00
train=train.loc[~(train['date_time']=='2011-01-01 00:00:00')].reset_index(drop=True)

### Data Features

In [ ]:
all_data = pd.concat([train, test])
# convert to datatime format
all_data['date_time'] = pd.to_datetime(all_data['date_time'])
all_data.head()

In [ ]:
#Year, Month, Day, Hour, Is_WeekEnd, SMC
all_data['Hour'] = all_data['date_time'].dt.hour
all_data["working_hours"] =  all_data["Hour"].isin(np.arange(8, 21, 1)).astype("int")
all_data["is_weekend"] = (all_data["date_time"].dt.dayofweek >= 5).astype("int")
all_data['satday'] = (all_data.date_time.dt.weekday==5).astype("int")
all_data['Hour'] = all_data.date_time.dt.hour*60+all_data.date_time.dt.minute
all_data['time'] = all_data['date_time'].astype(np.int64)//10**9
all_data["SMC"] = (all_data["absolute_humidity"] * 100) / all_data["relative_humidity"]

In [ ]:
import math

all_data['sensor_6'] = (all_data['sensor_2'] - all_data['sensor_5']) / all_data['sensor_5']
all_data['sensor_7'] = (all_data['sensor_3'] - all_data['sensor_4']) / all_data['sensor_4']
all_data['is_odd'] = (all_data['sensor_4'] < 646) & (all_data['absolute_humidity'] < 0.238)
all_data['day'] = all_data.date_time.dt.weekday

for periods in [3, 6]:
        all_data[f'dt-{periods}'] = all_data['deg_C'] - all_data['deg_C'].shift(periods=periods, 
                                                                                fill_value=0)
diff = all_data['date_time'] - min(all_data['date_time'])
trend = diff.dt.days

all_data['f1s'] = np.sin(trend * 2 * math.pi / (365 * 1)) 
all_data['f1c'] = np.cos(trend * 2 * math.pi / (365 * 1))
all_data['f2s'] = np.sin(2 * math.pi * trend / (365 * 2)) 
all_data['f2c'] = np.cos(2 * math.pi * trend / (365 * 2)) 
all_data['f3s'] = np.sin(2 * math.pi * trend / (365 * 3)) 
all_data['f3c'] = np.cos(2 * math.pi * trend / (365 * 3)) 
all_data['f4s'] = np.sin(2 * math.pi * trend / (365 * 4)) 
all_data['f4c'] = np.cos(2 * math.pi * trend / (365 * 4)) 
all_data['fh1s'] = np.sin(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 1))
all_data['fh1c'] = np.cos(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 1))
all_data['fh2s'] = np.sin(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 2))
all_data['fh2c'] = np.cos(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 2))
all_data['fh3s'] = np.sin(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 3))
all_data['fh3c'] = np.cos(diff.dt.seconds * 2 * math.pi / ( 3600 * 24 * 3))

In [ ]:
all_data['deg_C_is_odd'] = all_data['deg_C'].between(22.0, 26.5).astype('int')
all_data['sensor_3_is_odd'] = all_data['sensor_3'].between(1500, 2100).astype('int')

### Data Visualization

In [ ]:
train.hist(bins=15,figsize=(15,10),color='gray')
plt.show()

<div class="alert alert-block alert-info"> 📌 Intepretations.<br>
1.    Sensor data for Sensor 1, 2 and 3 is positively skewed.<br>
2.     So is the Target data<br>
3.     You also see a peaks in the Sensor data.<br></div>

In [ ]:
from scipy import stats
#Corelation of the Target Variable.
target=['target_benzene','target_carbon_monoxide','target_nitrogen_oxides']
plt.figure(figsize = (10, 8))

for t in target:
    fig, ax = plt.subplots(figsize=(8,4))
    plt.xlabel(t); plt.ylabel(t + 'Target Density'); 
    plt.title('Distribution of Target');
    
    sns.kdeplot(train[t] , label = t,color="black",)
    
    plt.grid()
    plt.show()
    
    fig, ax = plt.subplots(figsize=(8,4))
    res = stats.probplot(train[t],plot=plt)
    plt.show()
    

<div class="alert alert-block alert-info"> 📌 Intepretations.<br>
    1. We Observe that the target variables shows peakedness and a positive skew.<br>
    2. It would be good to apply a Log transformation to this data.<br></div>

In [ ]:
# Reference : https://www.kaggle.com/anjalianupam/tps-july-eda

target=['target_benzene','target_carbon_monoxide','target_nitrogen_oxides']
color=['red','black','blue']
i = 0
for t in target:
    my_alpha=0.25
    fig, ax = plt.subplots(figsize=(16,4))
    ax.scatter(all_data['date_time'].apply(lambda x: str(x).split(" ")[0]), all_data[t], alpha=my_alpha, 
               color=color[i], s=4)
    ax.xaxis.set_major_locator(plt.MaxNLocator(20)) # reduce number of x-axis labels
    plt.xticks(rotation=90)
    plt.title(t)
    plt.grid()
    plt.show()
    i+=1

<div class="alert alert-block alert-info"> 📌 Intepretations.<br>
    1. There appears to be a corelation of the peaks and lows with months.<br>
    2. Later part of the year Nitrogen Oxides significantly rise<br>
    3. There is also a notable dip in the month of August across all target variables<br></div>

In [ ]:
#correlation matrix
corrmat = train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

<div class="alert alert-block alert-info"> 📌 Intepretations.<br>
    1. Sensor 3 data appears to have a high negative corelation with teh target variable and other sensor data.<br>
    2. Sensor 1, 2 and 5 appears to have a positive corelation with the target and other Sensor data
    <br></div>

## Split Train and Test

In [ ]:
# Split train data
months = all_data["date_time"].dt.month[:len(train)]
all_data.drop(columns = 'date_time', inplace = True)
train_data = all_data.loc[~all_data.target_nitrogen_oxides.isnull()].drop(columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
y = all_data.loc[~all_data.target_nitrogen_oxides.isnull()][['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]

y_log=np.log1p(y)

# Split test data
test_data = all_data.loc[all_data.target_nitrogen_oxides.isnull()].drop(columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
train_data.shape,y.shape,test_data.shape,all_data.shape

In [ ]:
#histogram and normal probability plot
from scipy.stats import norm
target=['target_benzene','target_carbon_monoxide','target_nitrogen_oxides']
for t in target:
    fig = plt.figure()
    sns.distplot(y_log[t], fit=norm);
    res = stats.probplot(y_log[t], plot=plt)

In [ ]:
train_data.columns

In [ ]:
# cols = ['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1', 'sensor_2', 'sensor_3', 
#  'sensor_4', 'sensor_5', 'working_hours', 'is_weekend', 'time']

cols = ['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5','sensor_6', 'sensor_7',
        'is_odd',
        #'deg_C_is_odd', 'sensor_3_is_odd',
       'dt-3', 'dt-6', 'f1s', 'f1c',
       'f2s', 'f2c', 'f3s', 'f3c', 'f4s', 'f4c', 
        'fh1s', 'fh1c', 'fh2s','fh2c', 'fh3s', 'fh3c',
        'is_weekend', 'working_hours','time', 'SMC',]

# train_data = train_data[cols]
# test_data = test_data[cols]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(train_data[cols])
train_data_s = scaler.transform(train_data[cols])
test_data_s = scaler.transform(test_data[cols])

<div class="alert alert-block alert-info"> 📌 Intepretations.<br>
    1. As we have noted above scaling the data will be appropriate.<br>
    2. We have also applied a Log Transformation to the target variable<br></div>

In [ ]:
# https://www.kaggle.com/andy6804tw/catboost-13feature-cross-validation
cb_params = [
                {'learning_rate': 0.010169009412219588,
                 'l2_leaf_reg': 8.908337085912136,
                 'bagging_temperature': 8.384477224270551,
                 'random_strength': 1.950237493637981,
                 'depth': 6,
                 'grow_policy': 'Lossguide',
                 'leaf_estimation_method': 'Newton'},
                {'learning_rate': 0.166394867169309,
                 'l2_leaf_reg': 8.704675157564441,
                 'bagging_temperature': 3.340826164726799,
                 'random_strength': 1.538518016574368,
                 'depth': 2,
                 'grow_policy': 'Depthwise',
                 'leaf_estimation_method': 'Newton'},
                {'learning_rate': 0.028141156076957437,
                 'l2_leaf_reg': 3.116523267336638,
                 'bagging_temperature': 4.420661209459851,
                 'random_strength': 1.8011752694610028,
                 'depth': 6,
                 'grow_policy': 'Depthwise',
                 'leaf_estimation_method': 'Newton'},
            ]

In [ ]:
model_preds = 0
target_names=y_log.columns

# for i, target in enumerate(target_names):
#     print(f"\nTraining for {target}...")
classifier = ClassifierChain(CatBoostRegressor(random_state=2021,
                                 thread_count=4,
                                 verbose=False,
                                 loss_function='RMSE',
                                 eval_metric='RMSE',
                                 od_type="Iter",
                                 early_stopping_rounds=500,
                                 #use_best_model=True,
                                 iterations=10000,
                                 **cb_params[0]))

classifier.fit(train_data_s, y_log.values)

model_preds = np.expm1(pd.DataFrame.sparse.from_spmatrix(classifier.predict(test_data_s)))        

In [ ]:
preds = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')
preds[target_names] = model_preds

In [ ]:
preds.to_csv('submission.csv', index=False)

In [ ]:
# finish

## 👍 UpVote if you find helpful.